In [2]:
### Ligand file creation - how exactly do we generate the SDF files for novel lignands?

import os
import subprocess
from SBLMDCOVDOCK.SBLSettings import GROMACS_Settings
import requests
settings = GROMACS_Settings()

In [3]:
# use: https://ambermd.org/tutorials/basic/tutorial5/index.php
# only partially works with GAFF - doesnt generate charges.

resname = "KCX"

# download from PDB
# 
link = f"https://files.rcsb.org/ligands/download/{resname}.cif"

# download the file to KCX_dir

KCX_dir = os.path.join(settings.structures_input, resname)

# Check if the directory exists, if not, create it
if not os.path.exists(KCX_dir):
    os.makedirs(KCX_dir)

# Download the CIF file and save it to the target directory
cif_file_path = os.path.join(KCX_dir, f"{resname}.cif")
response = requests.get(link)

if response.status_code == 200:
    with open(cif_file_path, "wb") as f:
        f.write(response.content)
    print(f"CIF file downloaded and saved to {cif_file_path}")
else:
    print("Failed to download the CIF file.")

CIF file downloaded and saved to start_structures/APO/KCX/KCX.cif


In [33]:
# copy APO_1K55_chimeraFH.pdb into KCX
import shutil
pdb_orig = "APO_1K55_chimeraFH.pdb"

source_path = os.path.join(settings.structures_input, pdb_orig)
dest_path = os.path.join(KCX_dir, pdb_orig)
dest = shutil.copy(source_path, dest_path)
print(dest)

pdb_final = "1K55_4amber.pdb"

pdb4amber_command = [
    "pdb4amber",
    "-i", pdb_orig,
    "-o", pdb_final,

]

# Run pdb4amber
print(" ".join(pdb4amber_command))
subprocess.run(pdb4amber_command, cwd=KCX_dir)

start_structures/APO/KCX/APO_1K55_chimeraFH.pdb
pdb4amber -i APO_1K55_chimeraFH.pdb -o 1K55_4amber.pdb



Summary of pdb4amber for: APO_1K55_chimeraFH.pdb

----------Chains
The following (original) chains have been found:
A

---------- Alternate Locations (Original Residues!))

The following residues had alternate locations:
LYS_91
MET_99
SER_147
VAL_211
-----------Non-standard-resnames
KCX

---------- Gaps (Renumbered Residues!)
gap of 3.039053 A between PHE 49 and ILE 51

---------- Missing heavy atom(s)

None
The alternate coordinates have been discarded.
Only the first occurrence for each atom was kept.


CompletedProcess(args=['pdb4amber', '-i', 'APO_1K55_chimeraFH.pdb', '-o', '1K55_4amber.pdb'], returncode=0)

In [50]:


print(KCX_dir)

KCX_cif_path = os.path.join(KCX_dir, resname+'.cif')

antechamber_command = [
    'antechamber',
    '-i', KCX_cif_path,
    '-fi', 'ccif',
    "-bk", resname, # component block for ccif
    '-o', KCX_cif_path.replace('.cif', '.ac'),
    '-fo', 'ac',
    '-c', 'bcc', # charge method
    '-at', 'amber', # ff atom types
    '-s', '2', # verbose
    '-nc', '0'

]

print(" ".join(antechamber_command))

subprocess.run(antechamber_command, check=True)


Running: /Users/alexi/miniconda/envs/RIN_test/bin/am1bcc -i ANTECHAMBER_AM1BCC_PRE.AC -o ANTECHAMBER_AM1BCC.AC -f ac -p /Users/alexi/miniconda/envs/RIN_test/dat/antechamber/BCCPARM.DAT -s 2 -j 1

Running: /Users/alexi/miniconda/envs/RIN_test/bin/atomtype -f ac -p bcc -o ANTECHAMBER_AM1BCC.AC -i ANTECHAMBER_AM1BCC_PRE.AC

start_structures/APO/KCX
antechamber -i start_structures/APO/KCX/KCX.cif -fi ccif -bk KCX -o start_structures/APO/KCX/KCX.ac -fo ac -c bcc -at amber -s 2 -nc 0

Welcome to antechamber 22.0: molecular input file processor.

Info: acdoctor mode is on: check and diagnose problems in the input file.
Info: The atom type is set to amber; the options available to the -at flag are
      gaff, gaff2, amber, bcc, and sybyl.

Info: Determining atomic numbers from atomic symbols which are case sensitive.
-- Check Unusual Elements --
   Status: pass
-- Check Open Valences --
   Status: pass
-- Check Geometry --
      for those bonded   
      for those not bonded   
   Status: pas

/Users/alexi/miniconda/envs/RIN_test/bin/wrapped_progs/antechamber: Non Fatal Error!
Residue (KCX) has a type of LINKING.
    This linking monomer will likely become part of a polymer
    and thus probably requires special handling beyond antechamber.
Ignore this error only if you know what you are doing.




Running: /Users/alexi/miniconda/envs/RIN_test/bin/am1bcc -i ANTECHAMBER_AM1BCC_PRE.AC -o ANTECHAMBER_AM1BCC.AC -f ac -p /Users/alexi/miniconda/envs/RIN_test/dat/antechamber/BCCPARM.DAT -s 2 -j 1

Running: /Users/alexi/miniconda/envs/RIN_test/bin/atomtype -f ac -p bcc -o ANTECHAMBER_AM1BCC.AC -i ANTECHAMBER_AM1BCC_PRE.AC



CompletedProcess(args=['antechamber', '-i', 'start_structures/APO/KCX/KCX.cif', '-fi', 'ccif', '-bk', 'KCX', '-o', 'start_structures/APO/KCX/KCX.ac', '-fo', 'ac', '-c', 'bcc', '-at', 'amber', '-s', '2', '-nc', '0'], returncode=0)

In [51]:
# change NT to N in KCX.ac file
KCX_ac_path = KCX_cif_path.replace('.cif', '.ac')

# Read the contents of the KCX.ac file
with open(KCX_ac_path, 'r') as f:
    ac_contents = f.read()

# Perform the replacement (NT to N)
ac_contents = ac_contents.replace("NT", "N")

# Write the updated contents back to the KCX.ac file
with open(KCX_ac_path, 'w+') as f:
    # print(ac_contents)
    f.write(ac_contents)



In [65]:
KCX_mc = """
HEAD_NAME N
TAIL_NAME C
MAIN_CHAIN CA
MAIN_CHAIN CB
MAIN_CHAIN CG
MAIN_CHAIN CD
MAIN_CHAIN CE
MAIN_CHAIN NZ
MAIN_CHAIN CX
MAIN_CHAIN OQ1
MAIN_CHAIN OQ2
OMIT_NAME HN2
OMIT_NAME OXT
OMIT_NAME HXT
OMIT_NAME HQ2
PRE_HEAD_TYPE C
POST_TAIL_TYPE N
CHARGE -1.0
"""
# Neutral only
# KCX_mc = """
# HEAD_NAME N
# TAIL_NAME C
# MAIN_CHAIN CA
# MAIN_CHAIN CB
# MAIN_CHAIN CG
# MAIN_CHAIN CD
# MAIN_CHAIN CE
# MAIN_CHAIN NZ
# MAIN_CHAIN CX
# MAIN_CHAIN OQ1
# MAIN_CHAIN OQ2
# OMIT_NAME HN2
# OMIT_NAME OXT
# OMIT_NAME HXT
# PRE_HEAD_TYPE C
# POST_TAIL_TYPE N
# CHARGE 0
# """

path = os.path.join(KCX_dir, resname+'.mc')
print(path)
with open(path, 'w') as f:
    f.write(KCX_mc)



start_structures/APO/KCX/KCX.mc


In [66]:

prepgen_command = [
    "prepgen",
    "-i", resname+'.ac',
    "-o", resname+'.prepin',
    "-m", resname+'.mc',
    "-rn", resname,
]
    
print(" ".join(prepgen_command))
subprocess.run(prepgen_command, check=True, cwd=KCX_dir)

prepgen -i KCX.ac -o KCX.prepin -m KCX.mc -rn KCX

PRE_HEAD_TYPE is     C
POST_TAIL_TYPE is     N
Net charge of truncated molecule is     0.00
HEAD_ATOM      1    N
TAIL_ATOM      8    C
MAIN_CHAIN     1    1    N
MAIN_CHAIN     2    2   CA
MAIN_CHAIN     3    3   CB
MAIN_CHAIN     4    4   CG
MAIN_CHAIN     5    5   CD
MAIN_CHAIN     6    6   CE
MAIN_CHAIN     7    7   NZ
MAIN_CHAIN     8   10   CX
MAIN_CHAIN     9   12  OQ1
MAIN_CHAIN    10   13  OQ2
MAIN_CHAIN    11    8    C
OMIT_ATOM      1   15  HN2
OMIT_ATOM      2   11  OXT
OMIT_ATOM      3   26  HXT
Number of mainchain atoms (including head and tail atom):    11
Number of omited atoms:     3

CompletedProcess(args=['prepgen', '-i', 'KCX.ac', '-o', 'KCX.prepin', '-m', 'KCX.mc', '-rn', 'KCX'], returncode=0)

In [67]:
# find
!open $AMBERHOME

In [68]:
# adapt parmchk2 -i cro.prepin -f prepi -o frcmod.cro -a Y -p $AMBERHOME/dat/leap/parm/parm10.dat

ff_dat = "parm19.dat"

parmchk_command = [
    "parmchk2",
    "-i", resname + ".prepin",
    "-f", "prepi",
    "-o", "frcmod." + resname,
    "-a", "Y",
    "-p", os.environ["AMBERHOME"]+"/dat/leap/parm/"+ ff_dat
]

print(" ".join(parmchk_command))
subprocess.run(parmchk_command, check=True, cwd=KCX_dir)

parmchk2 -i KCX.prepin -f prepi -o frcmod.KCX -a Y -p /Users/alexi/miniconda/envs/RIN_test/dat/leap/parm/parm19.dat


CompletedProcess(args=['parmchk2', '-i', 'KCX.prepin', '-f', 'prepi', '-o', 'frcmod.KCX', '-a', 'Y', '-p', '/Users/alexi/miniconda/envs/RIN_test/dat/leap/parm/parm19.dat'], returncode=0)

In [69]:
# strip out ATTN lines

# ! grep -v "ATTN" {"frcmod."+resname} > {"frcmod1."+resname} # Strip out ATTN lines

grep_command = [
    "grep", "-v", "ATTN",
    os.path.join(KCX_dir, "frcmod."+resname),
    ">", os.path.join(KCX_dir, "frcmod1."+resname)
]

# Run grep command
print(" ".join(grep_command))
! {" ".join(grep_command)}
# subprocess.run(grep_command, check=True)

grep -v ATTN start_structures/APO/KCX/frcmod.KCX > start_structures/APO/KCX/frcmod1.KCX


In [70]:
# GAFF parameters for ligands
# adapt parmchk2 -i cro.prepin -f prepi -o frcmod.cro 


parmchk_command = [
    "parmchk2",
    "-i", resname + ".prepin",
    "-f", "prepi",
    "-o", "frcmod2." + resname,
    # "-a", "Y",
    # "-p", os.environ["AMBERHOME"]+"/dat/leap/parm/"+ ff_dat
]

print(" ".join(parmchk_command))
subprocess.run(parmchk_command, check=True, cwd=KCX_dir)

parmchk2 -i KCX.prepin -f prepi -o frcmod2.KCX


CompletedProcess(args=['parmchk2', '-i', 'KCX.prepin', '-f', 'prepi', '-o', 'frcmod2.KCX'], returncode=0)

In [71]:
pdb = pdb_orig
leap_file = f"""source leaprc.protein.ff19SB
set default PBRadii mbondi3
loadAmberPrep {resname}.prepin
loadAmberParams frcmod2.{resname}
loadAmberParams frcmod1.{resname}
x = loadPDB {pdb}
saveAmberParm x {pdb.replace(".pdb","_ante.parm7")} {pdb.replace(".pdb","_ante.rst7")}
quit
"""

leap_path = os.path.join(KCX_dir, 'tleap.in')

with open(leap_path, 'w') as f:
    f.write(leap_file)

In [72]:
tleap_command = [
    "tleap",
    "-f", "tleap.in" #os.path.join(KCX_dir, "tleap.in")
]
# subprocess.run("ls", cwd=KCX_dir)

# print(" ".join(tleap_command))
subprocess.run(tleap_command, cwd=KCX_dir)

-I: Adding /Users/alexi/miniconda/envs/RIN_test/dat/leap/prep to search path.
-I: Adding /Users/alexi/miniconda/envs/RIN_test/dat/leap/lib to search path.
-I: Adding /Users/alexi/miniconda/envs/RIN_test/dat/leap/parm to search path.
-I: Adding /Users/alexi/miniconda/envs/RIN_test/dat/leap/cmd to search path.
-f: Source tleap.in.

Welcome to LEaP!
(no leaprc in search path)
Sourcing: ./tleap.in
----- Source: /Users/alexi/miniconda/envs/RIN_test/dat/leap/cmd/leaprc.protein.ff19SB
----- Source of /Users/alexi/miniconda/envs/RIN_test/dat/leap/cmd/leaprc.protein.ff19SB done
Log file: ./leap.log
Loading parameters: /Users/alexi/miniconda/envs/RIN_test/dat/leap/parm/parm19.dat
Reading title:
PARM99 + frcmod.ff99SB + frcmod.parmbsc0 + OL3 for RNA + ff19SB
Loading parameters: /Users/alexi/miniconda/envs/RIN_test/dat/leap/parm/frcmod.ff19SB
Reading force field modification type file (frcmod)
Reading title:
ff19SB AA-specific backbone CMAPs for protein 07/25/2019
Loading library: /Users/alexi/min

CompletedProcess(args=['tleap', '-f', 'tleap.in'], returncode=31)